In [18]:
import urllib.request
import json
import pandas as pd
import requests
import shutil
import os
from PIL import Image
import numpy
import math
import matplotlib.pyplot as plot
from sklearn.cluster import KMeans



url = (
    "https://pixabay.com/api/?key=42732485-b4c1847a3529ea910e8e5fdeb&q=animaux&image_type=photo&per_page=100"
)
response = urllib.request.urlopen(url)
responsedata = json.loads(response.read().decode("utf-8"))
responsedata

def download_image(url):

    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        image_link = os.path.basename(url)
        filename = os.path.join("./images/", image_link)
        with open(filename, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code

array = []

for result in responsedata["hits"]:
    array.append(
        (
            result["id"],
            result["tags"],
            result["largeImageURL"],
            result["imageWidth"],
            result["imageHeight"],
            result["imageSize"],
            result["likes"], 
            result["user"],
            result["largeImageURL"].split('.')[-1]
        )
    )


dataframe = pd.DataFrame(array, columns=["id", "tags", "largeImageURL", "imageWidth","imageHeight","imageSize", "likes", "user", "format"])
dataframe = dataframe.astype(
    dtype={"id": "<U200", "tags": "<U200", "largeImageURL": "<U200","imageWidth": "<U200","imageHeight": "<U200","imageSize": "<U200", "likes": "<U200", "user": "<U200", "format": "<U200"}
)

# Color column
all_images=os.listdir("./images") #list of all the images' names
image_color=[] #list to add the images' color into the dataframe
dataframe['color']='None'
colorsOfAllImages=[] 
for i in range(len(all_images)):
    colorsOfOneImage="" #Reinitialization of the value of the colors
    color = [] #most significant colors of the images
    imgfile = Image.open("./images/"+all_images[i])
    numarray = numpy.array(imgfile.getdata(), numpy.uint8)
    if numarray.ndim == 1:
        imgfile = imgfile.convert('RGB')
        numarray = numpy.array(imgfile.getdata(), numpy.uint8)
    clusters = KMeans(n_clusters=4, n_init=2)
    clusters.fit(numarray)
    npbins = numpy.arange(0, 5)
    histogram = numpy.histogram(clusters.labels_, bins=npbins)
    labels = numpy.unique(clusters.labels_)
    for i in range(4):
        image_color=[math.ceil(clusters.cluster_centers_[i][0]),math.ceil(clusters.cluster_centers_[i][1]),math.ceil(clusters.cluster_centers_[i][2])]
        # Categorization
        for j in range(3):
            image_color[j] -= (image_color[j]) % 32
        color.append(image_color)
    colorsOfAllImages.append(color)
dataframe.loc[:,'color']=colorsOfAllImages

# Data type conversion to sort
for int_column in ["id","imageWidth","imageHeight","imageSize","likes"]:
    dataframe[int_column] = dataframe[int_column].astype(int)
    
# imageSize Categorization
for value in dataframe["imageSize"]:
    value -= value % 500000 

# Json conversion, Storage
json_data = dataframe.to_json(orient='records', indent = 4)
file_path = "data.json"
with open(file_path, "w") as json_file:
    json_file.write(json_data)

# Read test for the json produced
readtest_data = json.load(open("data.json"))
readtest_dataframe = pd.json_normalize(readtest_data)
readtest_dataframe

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

On va stocker id, tags, largeImageURL, imageWidth, imageHeight, imageSize, likes, format de l'image en prenant ce qu'il y a après le .; user

In [28]:

# Loads json
data = json.load(open("data.json"))
dataframe = pd.json_normalize(data)
#dataframe

# Sort
sort_key = input("Sort by : \n id / imageWidth / imageHeight / imageSize / color / likes ")
dataframe = dataframe.sort_values(by=sort_key)
# Json conversion, Storage
json_data = dataframe.to_json(orient='records', indent = 4)
file_path = "data.json"
with open(file_path, "w") as json_file:
    json_file.write(json_data)

def occurrences(column):
    res = {}
    for value in dataframe[column]:
        if value in res:
            res[value] += 1
        else:
            res[value] = 1
    return res


# imageSize Categorization
dataframe["imageSize"] -= dataframe["imageSize"] % 500000 
for i in range(len(dataframe["imageSize"])):
    if dataframe.loc[i, "imageSize"] > 2000000:
        dataframe.loc[i, "imageSize"] = 2000000
dataframe

print(occurrences("imageSize"))


{0: 2, 500000: 8, 1000000: 12, 1500000: 17, 2000000: 61}
